In [2]:
import requests 
import pandas as pd 
import json 
import numpy as np
import pprint
from bs4 import BeautifulSoup
import time
import numpy as np
import string
import re
import unidecode

In [14]:
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}

# testing web-scrape code to make sure it works

In [15]:
trial = 'https://stackoverflow.com/jobs/406677/senior-platform-engineer-paigeai?so=i&pg=2&offset=0'
trial_response = requests.get(trial, headers = header)
trial_soup = BeautifulSoup(trial_response.text, 'html.parser')

In [17]:
tech = []
job_tech = trial_soup.find_all('section', {'class':'mb32'})[1]('a')
for x in job_tech:
    tech.append(x.text)
    
tech

['amazon-web-services', 'kubernetes', 'git', 'python', 'powershell']

In [18]:
overview = ''
job_overview = trial_soup.find_all('section', {'class':'mb32'})[2](['p', 'ul'])
for y in job_overview:
    overview += y.text

overview

'Paige is a software company helping pathologists and clinicians make faster, more informed diagnostic and treatment decisions for patients with cancer by leveraging decades of data from the world’s experts in cancer care. We are leading a digital transformation in pathology with advanced Artificial Intelligence (AI) technology to create value for the oncology clinical team.\xa0Paige is the first company to develop clinical grade AI technology for the pathologist, which resulted in our receiving FDA breakthrough designation for our first product.We are seeking a\xa0Senior Platform Engineer\xa0who has experience building and scaling services in a cloud environment to join our Platform Engineering team.\xa0In this role you’ll collaborate with our AI and Software teams to automate and to streamline our production operations and processes. You will have the opportunity to work in a wide variety of areas including but not limited to advancing and pushing existing efforts in the areas of sec

In [116]:
position = str(trial_soup.find_all('h1', {'class':'fs-headline1 mb4'})[0].text)
type(position)

str

In [46]:
about = [trial_soup.find('section', {'class':'mb32'})('div')[1].text]
about

['\n\nJob type: \nFull-time\n\n\nExperience level: \nSenior\n\n\nRole: \nDevOps\n\n']

# creating looping functions to scrape website

In [48]:
# create an empty list to store all hyperlinks
all_hyperlinks = []

# function to retrieve link of every job posting
def get_hyperlinks(url, header):
    response = requests.get(url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')
    
# table holds the preview of every job posting
    job_table = soup.find_all('h2', {'mb4 fc-black-800 fs-body3'})
    
    for job in job_table:
#         retrieving every link
        job_link_html = job.find_all('a', href=True)
        
        for job_link in job_link_html:
            all_hyperlinks.append('https://stackoverflow.com'+ job_link['href'])
            
    return all_hyperlinks

In [49]:
# loop to go through every page and get a link for each job posting
# at the time of the web scrape, there were 116 pages
i = 1
while i <= 116:
    url = 'https://stackoverflow.com/jobs?pg={}'.format(i)        
    get_hyperlinks(url, header)
    i += 1

In [50]:
len(set(all_hyperlinks))

2893

In [51]:
len(all_hyperlinks)

2893

In [25]:
all_hyperlinks

['https://stackoverflow.com//jobs/260145/engineering-manager-force-therapeutics?a=1pfagBpGLmVi&so=i&pg=1&offset=0&total=2888&so_medium=Internal&so_source=JobSearch',
 'https://stackoverflow.com//jobs/406676/senior-software-engineer-backend-cloud-paigeai?a=2cnXVVvfrCms&so=i&pg=1&offset=1&total=2888&so_medium=Internal&so_source=JobSearch',
 'https://stackoverflow.com//jobs/218964/senior-nodejs-back-end-software-developer-prometheum-inc?a=1bqTr4axzdDy&so=i&pg=1&offset=2&total=2888&so_medium=Internal&so_source=JobSearch',
 'https://stackoverflow.com//jobs/403106/engineering-manager-simon-data?a=2bbJDBVzgg1O&so=i&pg=1&offset=3&total=2888&so_medium=Internal&so_source=JobSearch',
 'https://stackoverflow.com//jobs/398170/senior-backend-engineer-nyc-or-remote-mantl?a=29x6bU1gU5Co&so=i&pg=1&offset=4&total=2888&so_medium=Internal&so_source=JobSearch',
 'https://stackoverflow.com//jobs/368575/backend-engineer-simon-markets?a=1ZBJy5eHDxPG&so=i&pg=1&offset=5&total=2888&so_medium=Internal&so_source=J

In [55]:
# create an empty list to store dict of info for each job posting
job_desc_list = []

def each_job(url):
    
    time.sleep(2)
    
    response = requests.get(url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')
#     code for scraping technology required
#     making target variable out of this column, doing try & except so i can later drop row from df
    tech = []
    try:
        job_tech = soup.find_all('section', {'class':'mb32'})[1]('a')
        for x in job_tech:
            tech.append(x.text)
    except IndexError:
        tech = np.nan
#     code for scraping overview of the posting
    overview = ''
    try:
        job_overview = soup.find_all('section', {'class':'mb32'})[2](['p', 'ul'])
        for y in job_overview:
            overview += y.text
    except IndexError:
        overview = np.nan
#     code for scraping the job position
    try:
        position = [soup.find_all('h1', {'class':'fs-headline1 mb4'})[0].text]
    except IndexError:
        position = np.nan
#     code for brief insight
    try:
        about = [soup.find('section', {'class':'mb32'})('div')[1].text]
    except (IndexError, TypeError):
        about = np.nan
#     creating dictionary for each job posting
    job_post_dict = {
        'position':position,
        'description':about,
        'languages':tech,
        'overview':overview}
#     creating list of job postings to later turn into a dataframe
    job_desc_list.append(job_post_dict)
    
    return job_desc_list

In [56]:
# loop for every url in list
# set up counter to check for errors
s = 0
for link in all_hyperlinks:
    url = link
    each_job(url)
    s += 1

In [57]:
s

2893

In [13]:
job_desc_list[0]

{'position': ['\nEngineering Manager '],
 'description': ['\n\nJob type: \nFull-time\n\n\nExperience level: \nManager\n\n\nRole: \nFull Stack Developer\n\n'],
 'languages': ['python',
  'django',
  'javascript',
  'react',
  'amazon-web-services'],
 'overview': "Force is reimagining the way patients recover from injury and surgery. We are partnered with leading health systems across the United States to deliver prescriptive plans of care to their patients, measure multiple clinical data points and provide actionable insights into care and recovery. We have a direct and immediate feedback loop to our end users (patients and providers) and pride ourselves in being agile and responsive.What you’ll be doing: Our ideal engineering manager has a background and knowledge of functioning as an individual technical contributor working with web and mobile technologies. As a manager, you’ll work with engineers to make good technical decisions and support the career growth and development of the te

In [88]:
# loop to add url of job posting to each dict
for idx, value in enumerate(all_hyperlinks):
    job_desc_list[idx]['url'] = value

In [90]:
job_desc_list[0]

{'position': ['\nSoftware Engineer '],
 'description': ['\n\nJob type: \nFull-time\n\n\nRole: \nBackend Developer\n\n\nIndustry: \nFinancial Services\n\n'],
 'languages': ['java', 'api', 'python'],
 'overview': "This role requires a wide variety of strengths and capabilities, including:     * BS/BA degree in Computer Science or AI/Machine Learning     * Provide technical expertise throughout the software lifecycle including design, implementation and delivery     * Understanding of cloud, virtualization, APIs, and modern software languages     * Software development experience in one or more general-purpose programming languages: Python, Java, C, C++, Go     * Experience with developing frameworks that help increase developer and release velocity, improving code health and technical standards     * Advanced experience working with two or more from the following: web application development, database, unix/linux environments, distributed and parallel systems, information retrieval, netw

In [146]:
df = pd.DataFrame(job_desc_list)

In [163]:
# dropped any rows with nan values for languages since this will be my target variable
df.dropna(subset=['languages'], inplace=True)

In [176]:
df.shape

(2891, 4)

In [243]:
# column had empty lists as values, so i removed those as well
final = df[df.astype(str)['languages'] != '[]'].copy()

In [244]:
final.shape

(2366, 4)

# Cleaning df with regex

In [266]:
# creating regex condition for cleaning punctuation
remove = string.punctuation
remove = remove.replace('/', '') # don't remove hyphens
pattern = r'[{}]'.format(remove)

https://stackoverflow.com/questions/21209024/python-regex-remove-all-punctuation-except-hyphen-for-unicode-string/37708340

In [468]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''

    text = unidecode.unidecode(text)
    text.replace('\\n', '')
    text.strip(' \\n')
    text = re.sub(pattern, ' ', str(text))
    text = re.sub('(\\*n)', ' ', str(text))
    text = re.sub('\w*\d\w*', ' ', str(text))
    text = re.sub('  ', ' ', str(text))
    return text

In [469]:
clean = lambda x: clean_text(x)

In [470]:
final.position = pd.DataFrame(final.position.apply(clean))

In [471]:
final.description = pd.DataFrame(final.description.apply(clean))

In [472]:
final.overview = pd.DataFrame(final.overview.apply(clean))

In [550]:
# function to clean description column
def fix_description(text):
    separate = text.split()
    joined = ' '.join(list([x.strip('\\n') for x in d]))
    final_joined = ' '.join(hell.split('\\n')[::3])
    return final_joined

In [540]:
fix_description(final['description'][1])

'Job type Full time Experience level Senior Role Backend Developer DevOps'

In [546]:
final['clean_description'] = final.description.apply(lambda x: fix_description(x))

In [517]:
# function to retain position name
def keep_position_name(text):
    text = text[3:]
    return text

In [518]:
keep_position_name(final['position'][1])

'Senior Software Engineer Backend Cloud Computing '

In [545]:
final['position_title'] = final.position.apply(lambda x: keep_position_name(x))

In [548]:
# dropping for cleaned columns
final.drop('position', axis=1, inplace=True)
final.drop('description', axis=1, inplace=True)

In [640]:
final.head()

,languages,overview,position_title,clean_description,target
1,"[amazon-web-services, python, docker, kubernet...",Paige is a software company helping pathologis...,Senior Software Engineer Backend Cloud Computing,Job type Full time Experience level Senior Rol...,1
3,"[python, javascript, reactjs, django, typescript]",About UsSimon Data was founded in by a team of...,Engineering Manager,Job type Full time Experience level Senior Man...,1
4,"[node.js, kubernetes, microservices, typescrip...",Who is MANTL MANTL is a fast growing NYC based...,Senior Backend Engineer NYC Or Remote,Job type Full time Experience level Mid Level ...,0
5,"[api, postgresql, scala, amazon-web-services, ...",As a SIMON Backend Engineer you will play an i...,Backend Engineer,Job type Full time Experience level Mid Level ...,0
6,"[go, ruby, ruby-on-rails, redis, postgresql]",Gravity is looking for passionate software eng...,Software Engineer Backend NYC/LA,Job type Full time Experience level Junior Mid...,0


# creating target variable column

In [629]:
# created column for boolean result of languages column
final['almost'] = final['languages'].apply(lambda x: any(['python' in x for value in final['languages']]))

In [637]:
# created target variable column
targ_dict = {True:1, False:0}
final['target'] = final['almost'].map(targ_dict)

In [639]:
# dropping almost column now that i have target column
final.drop('almost', axis=1, inplace=True)

In [644]:
final.head()

,languages,overview,position_title,clean_description,target
1,"[amazon-web-services, python, docker, kubernet...",Paige is a software company helping pathologis...,Senior Software Engineer Backend Cloud Computing,Job type Full time Experience level Senior Rol...,1
3,"[python, javascript, reactjs, django, typescript]",About UsSimon Data was founded in by a team of...,Engineering Manager,Job type Full time Experience level Senior Man...,1
4,"[node.js, kubernetes, microservices, typescrip...",Who is MANTL MANTL is a fast growing NYC based...,Senior Backend Engineer NYC Or Remote,Job type Full time Experience level Mid Level ...,0
5,"[api, postgresql, scala, amazon-web-services, ...",As a SIMON Backend Engineer you will play an i...,Backend Engineer,Job type Full time Experience level Mid Level ...,0
6,"[go, ruby, ruby-on-rails, redis, postgresql]",Gravity is looking for passionate software eng...,Software Engineer Backend NYC/LA,Job type Full time Experience level Junior Mid...,0


In [646]:
# saving dataframe to csv
final.to_csv('job_post_dataframe.csv')

# creating dummy columns for programming languages

In [679]:
# creating dummy columns for all the programming languages
lang_dummy = final['languages'].str.join('|').str.get_dummies()

In [682]:
# code to check correlation of dummies compared to python
c = lang_dummy.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort")

In [731]:
nol = so['python'] > 0.09

In [736]:
nol.tail(11)
# flask, django, 

machine-learning    True
tensorflow          True
aws                 True
flask               True
go                  True
ruby                True
spring              True
c++                 True
r                   True
django              True
python              True
dtype: bool

In [739]:
# dropping python columns
lang_dummy.drop('python', axis=1, inplace=True)
# dropping columns with high correlation with python, associated with python, or don't make sense (like the last 2)
lang_dummy.drop(columns=['django', 'flask', 'matplotlib', 'pandas', 'python-3.x', 'pytest', 'matlab', 'pytorch', 
                         'scikit-learn', 'anaconda', 'Click here', 'here'])

,.net,.net-core,.netcore,3d,3dsmax,ab-testing,abap,accessibility,accommodations-ext@fb.com,actionscript,...,wpf,x64,xamarin,xapi,xbox-one,xcode,xcode-ui-testing,xml,zend-framework,zos
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [740]:
# saving dataframe of dummies of languages to csv
lang_dummy.to_csv('dummy_lang_df.csv')